# Basic Access of Magnitudes & Redshifts

Import necessary packages

In [1]:
from astroquery.sdss import SDSS

We will use the `SDSS.query_photoobj` function. This requires you to specify a run, rerun, camcol, and field parameter, and then followed by the fields that you want to select. Here is a summary of the individual numbers:
- `rerun` should always be 301 by default, no need to input this. If you visit https://dr17.sdss.org/sas/dr17/eboss/photoObj/, you see that `301` is the only option there anyway.
- `run` can be one of many values, visible when you click `301` at the link above.
- `camcol` can be one of the values `1,2,3,4,5,6`, again click through at the link above to see this.
- `field` is a value between around `11` and `800` or so. 

Read more about these values briefly here: https://www.sdss3.org/dr10/imaging/imaging_basics.php.

If you work at the granularity of each of these, you get a reasonable number of spectra. To illustrate, let's download one file for example. You could run

`wget https://dr17.sdss.org/sas/dr17/eboss/photoObj/301/1896/6/photoObj-001896-6-0042.fits`

to download the file corresponding to rerun 301, run 1896, camcol 6, field 42. This return information for about 1100 objects.

In [2]:
from astropy.io import fits
f = fits.open('/data/scratch/declan/scrphotoObj-001896-6-0042.fits')
len(f[1].data)

FileNotFoundError: [Errno 2] No such file or directory: '/data/scratch/declan/scrphotoObj-001896-6-0042.fits'

These files don't have the information we want really. The most important information they would have is the `RA` and `DEC` information that tell you where to find the actual images. We want the photometric magnitudes `u,g,r,i,z` for each "good" object. We can access this information using the function above.

In [3]:
result = SDSS.query_photoobj(run=1896, camcol=6, field=42, fields=['run', 'camcol', 'objid', 'u', 'g', 'r', 'i', 'z'])

In [3]:
result=SDSS.query_photoobj(299489677444933632)

In [8]:
print(result)

None


Unsurprisingly, the `run`, `camcol`, etc. fields aren't that useful, as they should be what we specified. 

In [5]:
result['run'][:10], result['camcol'][:10]

(<Column name='run' dtype='int64' length=10>
 1896
 1896
 1896
 1896
 1896
 1896
 1896
 1896
 1896
 1896,
 <Column name='camcol' dtype='int64' length=10>
 6
 6
 6
 6
 6
 6
 6
 6
 6
 6)

The other information is useful to us though. If you read the documentation at https://astroquery.readthedocs.io/en/latest/api/astroquery.sdss.SDSSClass.html#astroquery.sdss.SDSSClass.query_photoobj, you see this function queries a SQL table `photoObjALL` internal to SDSS. The fields available in this table should be visible here: https://skyserver.sdss.org/dr8/en/help/browser/description.asp?n=PhotoObjAll&t=U

(this documentation is for Data Releast 8, but there's not reason to believe stuff has changed much).

Let's do another query with more interesting features.

In [6]:
result = SDSS.query_photoobj(run=1896, camcol=6, field=42, 
                             fields=['specobjid', 'thingid', 'objid', 'clean', 'type',
                                     'ra', 'dec',
                                     'u', 'g', 'r', 'i', 'z'])

In [7]:
len(result)

1135

In [8]:
result[result['specobjid'] != 0]

specobjid,thingid,objid,clean,type,ra,dec,u,g,r,i,z
int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64


It appears that this particular combination did not yield any spectroscopic objects. In other words, we don't have redshift for any of these objects. Let's find a combination where we do have redshift values.

In [13]:
result = SDSS.query_photoobj(objid=627287639241811968,
                             fields=['specobjid', 'thingid', 'objid', 'clean', 'type',
                                     'ra', 'dec',
                                     'u', 'g', 'r', 'i', 'z'])

TypeError: SDSSClass.query_photoobj_async() got an unexpected keyword argument 'objid'

In [11]:
result[result['specobjid'] != 0]

TypeError: 'NoneType' object is not subscriptable

In [9]:
result = SDSS.query_photoobj(run=1896, camcol=4, field=44, 
                             fields=['specobjid', 'thingid', 'objid', 'clean', 'type',
                                     'ra', 'dec',
                                     'u', 'g', 'r', 'i', 'z'])
result[result['specobjid'] != 0]

specobjid,thingid,objid,clean,type,ra,dec,u,g,r,i,z
int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64
627287639241811968,517277208,1237653618005114976,1,3,148.906417527231,57.2878849881887,20.10997,18.25983,17.17231,16.7312,16.39058


Here we have a specobjid. The `clean` flag is 1, indicating that the photometry is clean, and we have the 5 magnitudes here.

Your task: write scripts that crawl the photometry directories (https://dr17.sdss.org/sas/dr17/eboss/photoObj/301/) to find such spectrographic objects as above. Merge on `specobjid` field of `specObj-dr17.fits` file that you're used to, to pair up redshift and photometry magnitude values. Ideally, we would save these into a single large data file for later access.

You can get bigger chunks by increasing the granularity level. This will probably speed up your scripts. But you don't want to increase it too much or it may overload, e.g.

In [60]:
result = SDSS.query_photoobj(run=1896, camcol=4, 
                             fields=['specobjid', 'thingid', 'objid', 'clean', 'type',
                                     'ra', 'dec',
                                     'u', 'g', 'r', 'i', 'z'])
result[result['specobjid'] != 0]

specobjid,thingid,objid,clean,type,ra,dec,u,g,r,i,z
int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64
9222493101293019136,516670228,1237653618004983854,0,6,148.663884107955,57.1556040396836,20.46411,20.24028,20.15152,19.70414,19.85128
9222482106176741376,516669808,1237653618004984067,1,6,148.588091886485,57.2222552984393,22.14285,21.80387,21.53887,21.46236,21.85623
9222477983008137216,516669964,1237653618004984080,1,6,148.394389214129,57.2140463979468,21.94235,21.62139,21.46745,21.30135,21.80009
9222481831298834432,516669699,1237653618004984081,1,6,148.533048369095,57.0979404803191,22.31532,21.90552,21.58582,21.58495,21.22673
9222479082519764992,516669966,1237653618004984091,0,6,148.4631591226,57.2338150493594,22.00823,21.78102,22.00979,21.65105,21.41146
9222478532763951104,516670108,1237653618004984488,1,3,148.331869504307,57.2566399966046,23.92012,22.46631,21.0765,20.04347,19.10344
7977137015694008320,516670012,1237653618004984240,1,3,148.424072702878,57.2039510894663,21.89005,21.79497,20.35818,19.60019,19.29862
627279667782510592,516669797,1237653618004983999,1,3,148.612406012918,57.1948228763001,19.53458,17.78387,16.88759,16.46856,16.16714
627277193881348096,516669950,1237653618004983997,1,3,148.568313473861,57.2032048184968,19.85847,18.00166,17.08681,16.66024,16.35817


In [61]:
result = SDSS.query_photoobj(run=1896, 
                             fields=['specobjid', 'thingid', 'objid', 'clean', 'type',
                                     'ra', 'dec',
                                     'u', 'g', 'r', 'i', 'z'])
result[result['specobjid'] != 0]

specobjid,thingid,objid,clean,type,ra,dec,u,g,r,i,z
float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64
7.980573264945586e+18,522080002,1237653616396730692,1,3,158.475674921478,58.5977990964425,21.71715,20.50551,19.3521,18.90891,18.66907
7.981541934756766e+18,522079936,1237653616396730438,1,3,158.639671673263,58.6254979222728,24.76101,20.34171,18.7709,18.14729,17.75275
7.977135641304474e+18,516669883,1237653618004983824,1,3,148.411950447632,57.2833454177062,24.16409,20.67602,19.02488,18.44317,18.1202
6.283129349084549e+17,516669684,1237653618004983893,1,3,148.477103122247,57.1369269017234,19.20603,17.16374,16.17346,15.70642,15.37886
6.272785682708828e+17,516669897,1237653618004983895,0,3,148.424676726107,57.1876602826736,18.64097,17.15355,16.38933,15.94962,15.66741
6.283151339317105e+17,516669898,1237653618004983898,1,3,148.452766972412,57.1735708737278,18.64733,16.62336,15.65729,15.2158,14.8937
6.283145841758966e+17,516669899,1237653618004983899,1,3,148.481175332892,57.1783091888119,19.39235,17.3369,16.36613,15.90162,15.56297
6.272769190034412e+17,516669900,1237653618004983900,0,3,148.466236781085,57.178576577917,19.54974,17.51906,16.35728,15.81327,15.37057
6.283154088096174e+17,516669930,1237653618004983942,1,3,148.448625886292,57.1996967879486,20.21815,18.42245,17.53723,17.11058,16.78622


Remember to filter for clean = 1 as well. If you iterate over runs, you should end up with a sizable number of objects overall. 